In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [4]:
db = 'database.sqlite'
query_fname = 'db_queries/detailed_match_query.sql'

conn = sqlite3.connect(db)

q_file = open(query_fname, 'r')
data = pd.read_sql(q_file.read(), conn)

data.head()

,id,country_name,league_name,season,stage,date,home_team,away_team,home_team_goal,away_team_goal
3035,4705,England,England Premier League,2015/2016,38,2016-05-15 00:00:00,Stoke City,West Ham United,2,1
3036,4706,England,England Premier League,2015/2016,38,2016-05-15 00:00:00,Swansea City,Manchester City,1,1
3037,4707,England,England Premier League,2015/2016,38,2016-05-15 00:00:00,Watford,Sunderland,2,2
3038,4708,England,England Premier League,2015/2016,38,2016-05-15 00:00:00,West Bromwich Albion,Liverpool,1,1
3039,4702,England,England Premier League,2015/2016,38,2016-05-17 00:00:00,Manchester United,Bournemouth,3,1


In [5]:
homes = pd.unique(data.home_team)
aways = pd.unique(data.away_team)

print(homes)
print(aways)
print(np.union1d(homes,aways))
teams = np.union1d(homes,aways)

['Arsenal' 'Sunderland' 'West Ham United' 'Everton' 'Middlesbrough'
 'Bolton Wanderers' 'Hull City' 'Manchester United' 'Aston Villa'
 'Chelsea' 'Liverpool' 'Tottenham Hotspur' 'Newcastle United'
 'Blackburn Rovers' 'Fulham' 'West Bromwich Albion' 'Stoke City'
 'Manchester City' 'Wigan Athletic' 'Portsmouth' 'Wolverhampton Wanderers'
 'Birmingham City' 'Burnley' 'Blackpool' 'Queens Park Rangers'
 'Swansea City' 'Norwich City' 'Reading' 'Southampton' 'Crystal Palace'
 'Cardiff City' 'Leicester City' 'Bournemouth' 'Watford']
['West Bromwich Albion' 'Liverpool' 'Wigan Athletic' 'Blackburn Rovers'
 'Tottenham Hotspur' 'Stoke City' 'Fulham' 'Newcastle United'
 'Manchester City' 'Portsmouth' 'Middlesbrough' 'Sunderland'
 'Bolton Wanderers' 'Hull City' 'Arsenal' 'Everton' 'Aston Villa'
 'West Ham United' 'Chelsea' 'Manchester United' 'Burnley'
 'Birmingham City' 'Wolverhampton Wanderers' 'Blackpool' 'Norwich City'
 'Swansea City' 'Queens Park Rangers' 'Southampton' 'Reading'
 'Cardiff City' '

In [6]:
from trueskill import Rating, quality_1vs1, rate_1vs1

team_ratings = {}
for team in teams:
    team_ratings[team] = Rating()

In [7]:
for row in data.itertuples():
    draw = False
    if row.home_team_goal > row.away_team_goal:
        team_win = row.home_team
        team_lose = row.away_team
    elif row.home_team_goal < row.away_team_goal:
        team_win = row.away_team
        team_lose = row.home_team
    else:
        team_win = row.home_team
        team_lose = row.away_team
        draw=True
    
    rating_win = team_ratings[team_win]
    rating_lose = team_ratings[team_lose]
    rating_win, rating_lose = rate_1vs1(rating_win, rating_lose, drawn=draw)
    team_ratings[team_win] = rating_win
    team_ratings[team_lose] = rating_lose
    

In [8]:
import itertools
import math

def win_probability(team1, team2):
    delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
    sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
    size = len(team1) + len(team2)
    ts = trueskill.global_env()
    denom = math.sqrt(size * (ts.beta * ts.beta) + sum_sigma)
    
    return ts.cdf(delta_mu / denom)

In [10]:
import trueskill
trueskill.TrueSkill(mu=25.0, sigma=8.333333333333334, beta=4.166666666666667, tau=0.08333333333333334, draw_probability=0.1)

trueskill.TrueSkill(mu=25.000, sigma=8.333, beta=4.167, tau=0.083, draw_probability=10.0%)

In [11]:
team1 = team_ratings['Fulham']
team2 = team_ratings['Manchester United']

print(team1)
print(team2)
print(win_probability([team1], [team2]))
print(win_probability([team2], [team1]))

print(team1.mu)

trueskill.Rating(mu=23.849, sigma=0.777)
trueskill.Rating(mu=27.845, sigma=0.775)
0.2525007922773095
0.7474992077226905
23.849333743729666


In [11]:
import tensorflow as tf
import edward as ed

/Users/vramesh/Google Drive/University/Sem7/Prob Prog/trueskill_augmented/trueskill_env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)
